In [ ]:
from scipy.io import loadmat
from dreaml import DataFrame as DF
from dreaml.server import start
import dreaml.transformations as trans
import numpy as np

data = np.genfromtxt('iris.txt', delimiter=',', dtype=None)
n = len(data)
X_train = np.zeros((n,4))
y_train = np.zeros((n,1))
for i in range(n):
    if data[i][4]=="Iris-setosa":
        y_train[i] = 0
    elif data[i][4]=="Iris-versicolor":
        y_train[i] = 1
    else: 
        y_train[i] = 2
    for j in range(4):
        X_train[i,j] = data[i][j]
        
df = DF()
start(df)
df["data/","input/raw/"].set_matrix(X_train)
df["data/","input/label/"].set_matrix(y_train)
df["centroids/","input/raw/"] = trans.KMPP(df["data/","input/raw/"],3)

In [ ]:
from dreaml.dataframe.transform import FigureTransform
from bokeh.plotting import figure

class ScatterIris(FigureTransform):
    def create_figure(self,target_df,i,j):
        self.i = i
        self.j = j
        p = figure(plot_width=400, plot_height=400)
        ohe = np.asarray(df["centroids/","input/raw/"].T.ohe.todense())
        centroids = df["centroids/","input/raw/"].get_matrix()
        self.colors = ['blue','green','red']
        self.names = [str(target_df.hash())+s for s in ['Iris-setos','Iris-versicolor','Iris-virginica']]
        self.centroid_names = ['centroid'+s for s in self.names]
        for k in range(3):
            p.circle(x=X_train[ohe[:,k],self.i],y=X_train[ohe[:,k],self.j],color=self.colors[k],name=self.names[k])
            p.cross(x=[centroids[k,self.i]],y=[centroids[k,self.j]],color=self.colors[k],name=self.centroid_names[k],size=20)
        
        return p
    def update(self,p):
        ohe = np.asarray(df["centroids/","input/raw/"].T.ohe.todense())
        centroids = df["centroids/","input/raw/"].get_matrix()
        rs = [p.select(dict(name=self.names[i])) for i in range(3)]
        cs = [p.select(dict(name=self.centroid_names[i])) for i in range(3)]
        for k in range(3):
            rs[k][0].data_source.data.update(x=X_train[ohe[:,k],self.i],y=X_train[ohe[:,k],self.j])
            cs[k][0].data_source.data.update(x=[centroids[k,self.i]],y=[centroids[k,self.j]])

In [ ]:
df["test/0","test/0"] = ScatterIris(1,2)

In [ ]:
# ScatterIris(1,2)
for i in range(3):
    for j in range(i+1,4):
        df["plot/"+str(i),"plot/"+str(j)] = ScatterIris(i,j)